In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 16.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=25f1a5664cf0dfb7aeb03ede71a77c6a8eb782a6b92bad390c91ac141d3ea5bc
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
from google.colab import files 
  
  
uploaded = files.upload()

Saving flights_small.csv to flights_small.csv


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [4]:
flights = spark.read.csv('flights_small.csv', header=True)

FINS aquí els 3 passets necessaris per tenir un Spark DataFrame

In [5]:
flights.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

Ens han promès "estil" SQL, veiem-ho

In [6]:
flights.createOrReplaceTempView("flights")

In [7]:
query = 'FROM flights SELECT * LIMIT 10'

Cuidado, si no omplim el Catalog no funciona!

In [8]:
flights10 = spark.sql(query)

In [9]:
flights10.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

Ara veurem com fer el camí de Spark DataFrame cap a Pandas Dataframe. Camí invers.

In [10]:
import pandas as pd

In [11]:
query2 = 'SELECT origin, dest, COUNT(*) as N FROM flights GROUP BY origin, dest'

In [12]:
flights_counts = spark.sql(query2)

Invoquem cap a Pandas

In [13]:
pd_counts = flights_counts.toPandas()

In [14]:
print(pd_counts.head())

  origin dest    N
0    SEA  RNO    8
1    SEA  DTW   98
2    SEA  CLE    2
3    SEA  LAX  450
4    PDX  SEA  144


Atenció: A partir d'aquí, imaginem que la tasca corre en un altre node

Tenim la vista "flights" al cataleg, anem a recuperar un Spark DataFrame que es podria dir flights sense problema però anomenarem flights2

In [15]:
flights2 = spark.table('flights')

In [16]:
flights2.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

Afegirem una columna nova amb la duració en hores del vol

In [17]:
flights2 = flights2.withColumn('duration_hours', flights2.air_time/60)

Comprovació que hi ha la nova columna

In [18]:
flights2.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|    duration_hours|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|               2.2|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|               6.0|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|              1.85|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|1.3833333333333333|
|2014|    3|  9|     754|  

Ara podem filtrar amb Filter() que és semblant a fer un WHERE de SQL

In [19]:
flights2.filter("air_time > 120").show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|    duration_hours|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|               2.2|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|               6.0|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|2.1166666666666667|
|2014|    1| 15|    1037|        7|    1352|        2|     WN| N646SW|    48|   PDX| DEN|     121|     991|  10|    37|2.0166666666666666|
|2014|    4| 19|    1236|  

El mateix filtre que el pas anterior amb una altra sintaxi, escull la que més t'agradi

In [20]:
flights2.filter(flights2.air_time > 120).show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|    duration_hours|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|               2.2|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|               6.0|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|2.1166666666666667|
|2014|    1| 15|    1037|        7|    1352|        2|     WN| N646SW|    48|   PDX| DEN|     121|     991|  10|    37|2.0166666666666666|
|2014|    4| 19|    1236|  

Repetim en 2 passos.
Obtindrem els vols de llarga distància.

In [21]:
long_flights1 = flights2.filter("distance > 2720")
long_flights2 = flights2.filter(flights2.distance > 2720)
long_flights1.show()
long_flights2.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-----------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|   duration_hours|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-----------------+
|2014|    3|  9|    2219|       -1|     644|      -11|     AA| N5CNAA|  1070|   SEA| MIA|     303|    2724|  22|    19|             5.05|
|2014|    1|  5|    2302|       57|     745|       65|     AA| N630AA|  1070|   SEA| MIA|     299|    2724|  23|     2|4.983333333333333|
|2014|    9|  6|    2336|        1|     811|       -5|     AA| N3JHAA|  1070|   SEA| MIA|     312|    2724|  23|    36|              5.2|
|2014|   11| 30|     105|        0|     929|      -11|     AA| N3APAA|   212|   SEA| MIA|     308|    2724|   1|     5|5.133333333333334|
|2014|    9| 21|    2327|       -8

Ens molesta veure tantes columnes en exemples anteriors? Cap problema, fem "selects"

In [22]:
# Primer grup de columnes
selected1 = flights2.select("tailnum","origin","dest")
# Segon grup de columnes (sintaxi alternativa)
temp = flights2.select(flights2.origin, flights2.dest, flights2.carrier)
# Primer Filtre
filterA = flights2.origin == "SEA"
# Segon Filtre
filterB = flights2.dest == "PDX"
# Encadenem filtres
selected2 = temp.filter(filterA).filter(filterB)
# Comprovacions
selected1.show(5)
selected2.show()

+-------+------+----+
|tailnum|origin|dest|
+-------+------+----+
| N846VA|   SEA| LAX|
| N559AS|   SEA| HNL|
| N847VA|   SEA| SFO|
| N360SW|   PDX| SJC|
| N612AS|   SEA| BUR|
+-------+------+----+
only showing top 5 rows

+------+----+-------+
|origin|dest|carrier|
+------+----+-------+
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     AS|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     AS|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
+------+----+-------+
only showing top 20 rows



Anem a fer una operació de transformació més elaborada que les anteriors

In [23]:
# Definim velicitat promig i li posem un ALIAS
avg_speed = (flights2.distance/(flights2.air_time/60)).alias("avg_speed")
speed1 = flights2.select("origin","dest","tailnum", avg_speed)
speed1.show(5)

+------+----+-------+------------------+
|origin|dest|tailnum|         avg_speed|
+------+----+-------+------------------+
|   SEA| LAX| N846VA| 433.6363636363636|
|   SEA| HNL| N559AS| 446.1666666666667|
|   SEA| SFO| N847VA|367.02702702702703|
|   PDX| SJC| N360SW| 411.3253012048193|
|   SEA| BUR| N612AS| 442.6771653543307|
+------+----+-------+------------------+
only showing top 5 rows



La mateixa transformació anterior estil SQL

In [24]:
speed2 = flights2.selectExpr("origin","dest","tailnum","distance/(air_time/60) as avg_speed")
speed2.show(5)

+------+----+-------+------------------+
|origin|dest|tailnum|         avg_speed|
+------+----+-------+------------------+
|   SEA| LAX| N846VA| 433.6363636363636|
|   SEA| HNL| N559AS| 446.1666666666667|
|   SEA| SFO| N847VA|367.02702702702703|
|   PDX| SJC| N360SW| 411.3253012048193|
|   SEA| BUR| N612AS| 442.6771653543307|
+------+----+-------+------------------+
only showing top 5 rows



Arriba el torn dels AGREGATS: EXERCICI PROPOSAT, analitza error i troba la solució per a cada origen busquem una cosa en concret


In [39]:
flights2.filter(flights2.origin == "SEA").groupBy('tailnum').max().show()

+-------+-------------------+
|tailnum|max(duration_hours)|
+-------+-------------------+
| N442AS|  5.466666666666667|
| N102UW|  4.566666666666666|
| N38451|  4.316666666666666|
| N73283| 3.8333333333333335|
| N513UA|  4.483333333333333|
| N36472|                1.5|
| N954WN| 2.2666666666666666|
| N388DA|               3.55|
| N516UA|  4.316666666666666|
| N8322X| 3.7333333333333334|
| N466SW| 2.6333333333333333|
|  N6700|               4.15|
| N607AS| 3.6666666666666665|
| N584AS|  5.816666666666666|
| N914WN| 2.4833333333333334|
| N654AW|  2.283333333333333|
| N336NW| 1.3333333333333333|
| N445WN| 2.1166666666666667|
| N622SW| 1.9666666666666666|
| N669DN| 2.6333333333333333|
+-------+-------------------+
only showing top 20 rows



In [40]:
flights2.filter(flights2.origin == "PDX").groupBy('tailnum','distance').min().show()

+-------+--------+-------------------+
|tailnum|distance|min(duration_hours)|
+-------+--------+-------------------+
| N36207|     991|               2.05|
| N4YDAA|    1616|               3.05|
| N795AS|    1009| 2.0833333333333335|
| N803AW|    1009| 2.0166666666666666|
| N534AS|    2562|  5.133333333333334|
| N202AA|    1616|                3.1|
| N7743B|    1482|  2.816666666666667|
| N601AW|    1009| 1.8333333333333333|
| N3JYAA|    1739|               3.35|
| N918SW|     834| 1.9333333333333333|
| N242WN|     991|                1.8|
| N822SK|     129| 0.5833333333333334|
| N615SW|     569|               1.45|
| N3771K|    2172|  3.966666666666667|
| N462AS|     834|                1.9|
| N64809|     991| 1.9333333333333333|
| N4XTAA|    1616|  3.283333333333333|
| N581SW|     116|                0.5|
| N7720F|    1482| 2.8666666666666667|
| N496UA|     991|  2.033333333333333|
+-------+--------+-------------------+
only showing top 20 rows



In [41]:
flights2.filter(flights2.origin == "SEA").groupBy('tailnum','distance','air_time').avg().show()

+-------+--------+--------+-------------------+
|tailnum|distance|air_time|avg(duration_hours)|
+-------+--------+--------+-------------------+
| N431AS|    1489|     180|                3.0|
| N565SW|     129|      40| 0.6666666666666666|
| N644AS|    1709|     210|                3.5|
| N161PQ|     954|     118| 1.9666666666666666|
| N622AS|     978|     135|               2.25|
| N219AG|    1368|     160| 2.6666666666666665|
| N779AS|    1448|     205| 3.4166666666666665|
| N794AS|    1448|     189|               3.15|
| N319AS|    1721|     207|               3.45|
| N408AS|    1533|     196| 3.2666666666666666|
| N556UA|    2306|     261|               4.35|
| N553AS|     867|     121| 2.0166666666666666|
| N569AS|     937|     126|                2.1|
| N767AS|     697|     102|                1.7|
| N607JB|     965|     132|                2.2|
| N6715C|    2182|     236|  3.933333333333333|
| N569AS|    1448|     190| 3.1666666666666665|
| N39450|    1874|     223|  3.716666666

Seguim fent agregats

In [46]:
by_plane = flights2.groupBy("tailnum")
by_plane.count().show()

+-------+-----+
|tailnum|count|
+-------+-----+
| N442AS|   38|
| N102UW|    2|
| N36472|    4|
| N38451|    4|
| N73283|    4|
| N513UA|    2|
| N954WN|    5|
| N388DA|    3|
| N567AA|    1|
| N516UA|    2|
| N927DN|    1|
| N8322X|    1|
| N466SW|    1|
|  N6700|    1|
| N607AS|   45|
| N622SW|    4|
| N584AS|   31|
| N914WN|    4|
| N654AW|    2|
| N336NW|    1|
+-------+-----+
only showing top 20 rows



In [49]:
by_origin = flights2.groupBy("origin")
by_origin.avg("duration_hours").show()

+------+-------------------+
|origin|avg(duration_hours)|
+------+-------------------+
|   SEA|  2.673935826752105|
|   PDX| 2.2852572080481233|
+------+-------------------+



EXERCICI: prova de fer JOINS entre els DATAFRAMES flights i flights2


In [52]:
flights2= spark.sql(query)
flightsJoin = flights10.join(flights2, on='tailnum',how='left')
flightsJoin.show()

+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+
|tailnum|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|flight|origin|dest|air_time|distance|hour|minute|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|flight|origin|dest|air_time|distance|hour|minute|
+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+
| N846VA|2014|   12|  8|     658|       -7|     935|       -5|     VX|  1780|   SEA| LAX|     132|     954|   6|    58|2014|   12|  8|     658|       -7|     935|       -5|     VX|  1780|   SEA| LAX|     132|     954|   6|    58|
| N559AS|2014|    1| 22|    1040|        5|    1505|        5|     AS|   851|   